This example notebooks shows the local execution of Prot2Comp and of the singular databases 

In [1]:
#General packages
import pandas as pd
from tqdm import tqdm
import os
import time
from CPIExtract import Prot2Comp

# Load in Required Datasets

In [ ]:
# Root data path
data_path = 'data/Databases/'

#Downloaded from BindingDB on 3/30/2023
file_path=os.path.join(data_path, 'BindingDB.csv')
BDB_data=pd.read_csv(file_path,sep=',',usecols=['CID', 'Ligand SMILES','Ligand InChI','BindingDB MonomerID','Ligand InChI Key','BindingDB Ligand Name','Target Name Assigned by Curator or DataSource','Target Source Organism According to Curator or DataSource','Ki (nM)','IC50 (nM)','Kd (nM)','EC50 (nM)','pH','Temp (C)','Curation/DataSource','UniProt (SwissProt) Entry Name of Target Chain','UniProt (SwissProt) Primary ID of Target Chain'],on_bad_lines='skip')

#Downloaded from STITCH on 2/22/2023
file_path=os.path.join(data_path, 'STITCH.tsv')
sttch_data=pd.read_csv(file_path,sep='\t')

#Downloaded from ChEMBL on 2/01/2024
file_path=os.path.join(data_path, 'ChEMBL.csv')
chembl_data=pd.read_csv(file_path,sep=',')

file_path=os.path.join(data_path, 'CTD.csv')
CTD_data=pd.read_csv(file_path,sep=',')

#Downloaded from DTC on 2/24/2023
file_path=os.path.join(data_path, 'DTC.csv')
DTC_data=pd.read_csv(file_path,sep=',',usecols=['CID', 'compound_id','standard_inchi_key','target_id','gene_names','wildtype_or_mutant','mutation_info','standard_type','standard_relation','standard_value','standard_units','activity_comment','pubmed_id','doc_type'])

#Downloaded from DrugBank on 3/2/2022
file_path=os.path.join(data_path, 'DB.csv')
DB_data=pd.read_csv(file_path, sep=',')

#Downloaded from DrugCentral on 2/25/2024
file_path=os.path.join(data_path, 'DrugCentral.csv')
DC_data=pd.read_csv(file_path, sep=',')


In [ ]:
dbs = {
    'bdb': BDB_data,
    'chembl': chembl_data,
    'stitch': sttch_data,
    'ctd': CTD_data,
    'dtc': DTC_data,
    'dc': DC_data,
    'db': DB_data
}

## Load in Protein List

In [ ]:
inputids = pd.read_csv('data/input/db_proteins.csv')

## Run Prot2Comp on Protein List

In [ ]:
times = {}

In [ ]:
P2C = Prot2Comp('local', dbs=dbs)

In [ ]:
output_path = 'data/output/'

In [ ]:
p2c_dat=pd.DataFrame()
p2c_fail=pd.DataFrame()
states = pd.DataFrame()

r=0
for h in tqdm(range(0, 10)):
    
    protid=inputids['HGNC ID'].iloc[h]
    start = time.time()
    try:
        [comp_dat, state]=P2C.prot_interactions(input_id=protid, pChEMBL_thres=0)
        end = time.time()
        times[h] = end-start
        p2c_dat=pd.concat([p2c_dat,comp_dat])
        states = pd.concat([states, state])
        print(f'{h} done')
    except:
        # Collects failed inputids
        print(f'{h} failed')
        p2c_fail.loc[r,"failed_id"]=protid #Collects failed inputids
        r=r+1
    
    #Saves a file after a certain number of loops for insurance
    check=h/50 #automatically saves progress when on loop number set here
    if (check-int(check)==0)==True:
        filename=os.path.join(output_path, 'P2C_iter_'+str(h)+'.csv')
        p2c_dat.to_csv(filename,sep=',') 

# Save completed dataframe to file
filename=os.path.join(output_path, 'P2C.csv')
p2c_dat.to_csv(filename,sep=',',index=False)

In [ ]:
p2c_dat

In [ ]:
states

# Run Prot2Comp on Single Compound

## Prot2Compt2Comp with all Databases

### Test time required

In [ ]:
#inputid='AAWZDTNXLSGCEK-WYWMIBKRSA-N'
import cProfile
protid='KLK1'

In [ ]:
cProfile.run('P2C.prot_interactions(protid)')

### Run pipeline

In [ ]:
inp = 'Q12809'

In [ ]:
P2C = Prot2Comp('local', dbs=dbs)

In [ ]:
[comp_dat,states]=P2C.prot_interactions(input_id=inp, pChEMBL_thres=0)

In [ ]:
states

In [ ]:
comp_dat

### Prot2Compt with select Databases

In [ ]:
#inputid='AAWZDTNXLSGCEK-WYWMIBKRSA-N'
inputid='KLK1'

In [ ]:
P2C = Prot2Comp('local', dbs=dbs)

In [ ]:
#The second argument is a string listing the databases needed.
#Underscore is required to separate the databases.
[comp_dat,states]=P2C.prot_interactions_select(inputid,'pc_db_ctd_bdb_sttch')

In [ ]:
comp_dat

In [ ]:
states

# Run each step/database separately (for troubleshooting)

In [1]:
from cpie.utils.identifiers import *
import os
import pandas as pd
from cpie.databases import *
from cpie import protein_identifiers
input_id = 'KLK1'
protids = protein_identifiers(input_id)

In [ ]:
protids

### Run PubChem only

In [ ]:
pubchem = PubChem()
[pc_dat, pc_state, pc_raw] = pubchem.compounds(protids)

In [ ]:
pc_dat

In [ ]:
pc_state

### Run ChEMBL only

In [ ]:
#Downloaded from ChEMBL on 2/01/2024
file_path=os.path.expanduser(os.getcwd()+'\\data\\Databases\\ChEMBL.csv')
chembl_data=pd.read_csv(file_path,sep=',')

In [ ]:
chembl = ChEMBL(database=chembl_data)
[chembl_dat, state, chembl_raw] = chembl.compounds(protids)

In [ ]:
chembl_dat

In [ ]:
state

### Run BindingDB only

In [ ]:
#Downloaded from BindingDB on 3/30/2023
file_path=os.path.expanduser(os.getcwd()+'\\data\\Databases\\BindingDB.tsv')
BDB_data=pd.read_csv(file_path,sep='\t',
                     usecols=['Ligand SMILES','Ligand InChI','BindingDB MonomerID','Ligand InChI Key','BindingDB Ligand Name','Target Name Assigned by Curator or DataSource','Target Source Organism According to Curator or DataSource','Ki (nM)','IC50 (nM)','Kd (nM)','EC50 (nM)','pH','Temp (C)','Curation/DataSource','UniProt (SwissProt) Entry Name of Target Chain','UniProt (SwissProt) Primary ID of Target Chain'],on_bad_lines='skip')

In [ ]:
bdb = BindingDB(database=BDB_data)
bdb_dat, bdb_state, bdb_raw = bdb.compounds(protids, 0)

In [ ]:
bdb_raw

In [ ]:
bdb_state

### Run STITCH only

In [ ]:
#Downloaded from STITCH on 2/22/2023
file_path=os.path.expanduser(os.getcwd()+'\\data\\Databases\\STITCH.tsv')
sttch_data=pd.read_csv(file_path,sep='\t')

In [ ]:
stitch = Stitch(database=sttch_data)
[stitch_dat, stitch_state, stitch_raw] = stitch.compounds(protids)

In [ ]:
stitch_raw

In [ ]:
stitch_dat

In [ ]:
stitch_state

### Run CTD only

In [ ]:
#Downloaded from CTD on 2/24/2023
file_path=os.path.expanduser(os.getcwd()+'\\data\\Databases\\CTD.csv')
CTD_data=pd.read_csv(file_path,sep=',')

In [ ]:
ctd = CTD(database=CTD_data)
[ctd_dat, ctd_state, ctd_raw] = ctd.compounds(protids) 

In [ ]:
ctd_dat

### Run DTC only

In [ ]:
#Downloaded from DTC on 2/24/2023
file_path=os.path.expanduser(os.getcwd()+'\\data\\Databases\\DTC.csv')
DTC_data=pd.read_csv(file_path,sep=',',usecols=['compound_id','standard_inchi_key','target_id','gene_names','wildtype_or_mutant','mutation_info','standard_type','standard_relation','standard_value','standard_units','activity_comment','pubmed_id','doc_type'])

In [ ]:
dtc = DTC(database=DTC_data)
[dtc_dat, dtc_state, dtc_raw] = dtc.compounds(protids, 0)

In [ ]:
dtc_state

In [ ]:
dtc_dat

### Run OTP only

In [ ]:
otp = OTP() 
[otp_dat, otp_state, otp_raw] = otp.compounds(protids)

In [ ]:
otp_state

In [ ]:
otp_raw

In [ ]:
otp_dat

### Run DrugCentral only

In [ ]:
#Downloaded from DrugCentral on 2/25/2024
file_path=os.path.expanduser(os.getcwd()+'\\data\\DrugCentral.csv')
DC_data=pd.read_csv(file_path, sep=',')

In [ ]:
dc = DrugCentral(database=DC_data)
[DC_dat, DC_state, DC_raw] = dc.compounds(input_protein=protids)

In [ ]:
DC_dat

In [ ]:
DC_state

### Run DrugBank only

In [ ]:
#Downloaded from DrugBank on 3/2/2022
file_path=os.path.expanduser(os.getcwd()+'\\data\\Databases\\DB.csv')
DB_data=pd.read_csv(file_path, sep=',')

In [ ]:
db = DB(database=DB_data)
[DB_dat, DB_state, DB_raw] = db.compounds(protids)

In [ ]:
DB_state

In [ ]:
DB_dat

In [ ]:
DB_dat